In [50]:
from Dataset import DatasetBuilder

start_directory = "C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF"
pdf_processor = DatasetBuilder(start_directory)
data = pdf_processor.process_directory()

data

,Title,Version,Date,Confidentiality,Perimeter,Investigation Number,page_content,Source
0,Investigation de la cellule 909 Dégainage,2.0,07/05/2019,Diffusion Limitée Orano,DEG,ELH-2018-022986,TABLES DES MATIERES\n1\nDOCUMENTS DE REFERENCE...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
1,Investigation cellule 951 – DEG – DM DEG 15-...,1.0,10/04/2019,Diffusion Limitée Orano,DEG,ELH-2019-014855,TABLES DES MATIERES\n1\nDOCUMENTS DE REFERENCE...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
2,Investigation cellule 952 – DM DEG-150014,1.0,08/04/2019,Diffusion Limitée Orano,DEG,ELH-2019-017580,TABLES DES MATIERES\n1\nDOCUMENTS DE REFERENCE...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
3,Investigation cellule 965 – DEG – DM DEG 15-...,1.0,06/05/2019,Diffusion Limitée Orano,DEG,ELH-2019-022027,TABLES DES MATIERES\n1\nDOCUMENTS DE REFERENCE...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
4,Investigation des cellules 906 – 907 – 908 par...,1.0,30/07/2019,Diffusion Limitée Orano,DEG,ELH-2019-027845,TABLES DES MATIERES\n1.\nDOCUMENTS DE REFERENC...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
...,...,...,...,...,...,...,...,...
91,INVESTIGATIONS DU PUISARD 588-15 DE LA SALLE 7...,1.0,06/02/2023,Diffusion Normale Orano,STE2,ELH-2023-005541,TABLES DES MATIERES \n1 \nTABLEAU DE REVISION ...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
92,INVESTIGATIONS ET INVENTAIRE DANS LES SALLES 8...,1.0,30/05/2023,Diffusion Limitée Orano,STE2,ELH-2023-026550,TABLES DES MATIERES \n1 \nTABLEAU DE REVISION ...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
93,INVESTIGATIONS DES CUVES RÉACTEURS 531-11 ET 5...,1.0,09/06/2023,Diffusion Limitée Orano,STE2,ELH-2023-029462,TABLES DES MATIERES \n1 \nTABLEAU DE REVISION ...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...
94,INVESTIGATIONS DU PUISARD EXTERIEUR 588-16 – A...,1.0,13/09/2023,Diffusion Normale Orano,STE2,ELH-2023-049187,TABLES DES MATIERES \n1 \nTABLEAU DE REVISION ...,C:\Users\gciprianherrera\Desktop\LLM\MVP_Chatb...


In [4]:

import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
model: str = "text-embedding-ada-002"


from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model="gpt-35-turbo",
    temperature=0,
)

In [4]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMB_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_EMB_API_KEY"),
    openai_api_version="2023-05-15",
    azure_deployment=os.getenv("AZURE_OPENAI_EMB_DEPLOYMENT_NAME"),
)

In [6]:
index_name: str = "complete_dataset"
vector_store: AzureSearch = AzureSearch(
    azure_search_key = os.getenv("AZURE_AI_SEARCH_API_KEY"),
    azure_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    index_name=index_name,
    embedding_function=embeddings.embed_query
)

In [7]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(data, page_content_column="page_content")
documents = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken
from langchain_chroma import Chroma

tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=5,
    length_function=tiktoken_len,
    separators=["\n", " ", ""])
splits = text_splitter.split_documents(documents)
vector_store.add_documents(documents=splits)
# vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [9]:
AZURE_AI_SEARCH_API_KEY=os.getenv("AZURE_AI_SEARCH_API_KEY")
AZURE_AI_SEARCH_ENDPOINT=os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_DEPLOYEMENT_ID=os.getenv("AZURE_AI_SEARCH_DEPLOYEMENT_ID")
AZURE_AI_SEARCH_INDEX_NAME=os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
AZURE_AI_SEARCH_SERVICE_NAME=os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")

In [1]:
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import AzureChatOpenAI

from langchain_community.retrievers import (
    AzureAISearchRetriever,   
)

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model="gpt-35-turbo",
    temperature=0,
)

index_name: str = "complete_dataset"
retriever = AzureAISearchRetriever(
    content_key="content", top_k=30, index_name=index_name
)

# query = "quel est l'interet de la cellulle 951"
query = "quelle est l'objectif du compte rendu de la cellule 951 DEG"
# query = "de quoi il s'agit l'investigaton dans la zone nord ouest"
# query = "de quoi il s'agit Investigation sur l’ensemble porte-lame cellule 904"
# query = "a quelle perimetre appartient la porte-lame cellule 904"
# query = "qu'elles sont les équipements investigués sur le périmètre DEG"
# query = "qu'elle est ledebit de dose de la 951 du DEG"
# query = "qu'elles sont les informations de la cellule 909 du DEG"
# query = "qu'elle est le volume boue de de la fosse 217-02 du DEG"
# query = "DDD des PE de boues sur la fosse 217-02 du DEG"
# query = "enseignements sur investigation de la cellule 959B du HADE"
# query = "quelles les informations sur l'état de la cellule 929A du périmètre HADE"

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use five sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [4]:
chain.invoke({"input": query})['answer']

"L'objectif du compte-rendu de la cellule 951 DEG est de présenter les résultats des investigations réalisées dans cette cellule du bâtiment Dégainage depuis la salle 707. Cette investigation est décrite dans la Gamme Opératoire citée dans les documents de référence et a été réalisée en Janvier 2019."

In [1]:
class Chatbot:
    def __init__(self):
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2023-05-15",
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            model="gpt-35-turbo",
            temperature=0,
        )

        self.index_name = "complete_dataset"
        self.retriever = AzureAISearchRetriever(
            content_key="content", top_k=30, index_name=self.index_name
        )

        system_prompt = (
            "Use the given context to answer the question. "
            "If you don't know the answer, say you don't know. "
            "Use five sentences maximum and keep the answer concise. "
            "Context: {context}"
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
        self.question_answer_chain = create_stuff_documents_chain(self.llm, prompt)
        self.chain = create_retrieval_chain(self.retriever, self.question_answer_chain)

    def query(self, user_query):
        response = self.chain.invoke({"input": query})['answer']
        return response

In [2]:
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import AzureChatOpenAI

from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

chatbot = Chatbot()
query = "quelle est l'objectif du compte rendu de la cellule 951 DEG"
# query = "de quoi il s'agit l'investigaton dans la zone nord ouest"
# query = "de quoi il s'agit Investigation sur l’ensemble porte-lame cellule 904"
# query = "qu'elles sont les équipements investigués sur le périmètre DEG"
# query = "qu'elle est ledebit de dose de la 951 du DEG"
# query = "qu'elles sont les informations de la cellule 909 du DEG"
# query = "qu'elle est la dimension de la boue de de la fosse 217-02 du DEG"
# query = "DDD des PE de boues sur la fosse 217-02 du DEG"
# query = "enseignements sur investigation de la cellule 959B du HADE"
# query = "quelles les informations sur l'état de la cellule 929A du périmètre HADE"
response = chatbot.query(query)
print(response)

L'objectif du compte-rendu de la cellule 951 DEG est de présenter les résultats des investigations réalisées dans cette cellule du bâtiment Dégainage depuis la salle 707.
